
# Bayesian modeling of minds, brains and behavior
---
**Overview.** This notebook allows you to run Python code in your browser. We can mix text and code in a single document. This iis intended to support the lectures allowing me to demo certain concepts in a hands on way. It is also designed for you to work through it slowly after lectures, reading the text and doing the exercises by playing with the plots or even altering or writing code if you want to. We have an emphasis on play. Mess around. "I wonder what happens if I change this?". Let's start with a simple example of a Gaussian distribution.

---

## Lecture 2

**Beliefs as distributions.** Here you can adjust the sliders to change the mean (μ) and standard deviation (σ) of this Gaussian distribution. The mean moves the distribution so that its mean is higher or lower. The standard deviation changes the spread, changing the uncertainty of the belief

**Exercise.** Click on code cell below and press play to run. Play around with the distributions and think about how they would represent belief. Use the sliders to generate distributions that represent different prior beliefs for theta:

  - Certain belief that the theta is high

  - Certain belief that the theta is low 
  
  - Uncertain belief that the theta is high   
  
  - Uncertain belief that the theta is low*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian(mu=0.0, sigma=1.0):
    # Fixed x range
    x = np.linspace(-20, 20, 1000)
    y = norm.pdf(x, mu, sigma)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')
    ax.fill_between(x, y, color='plum', alpha=0.4)
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(-20, 20)
    ax.set_ylim(0, 3)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ")
);

interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

**Interpreting probability distributions.** There are intuitive ways to read off probabilities from probability distributions like these. According to the belief encoded by the distribution, the probability of theta being larger than say 0.5 or 5 is the area under the curve above this value. Similarly, the probability of theta being between say 0.1 and 0.7 is the area under the curve between these two values

**Exercise.** Click on the code cell below and press play to run. 

- Play with the sliders to see how the area under the curve changes as you change the minimum and maximum values. 

- The shaded area represents the probability of theta being in the range you selected.

- What happens when you make the distribution more narrow, or more wide? 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian(mu=0.0, sigma=1.0, x_min=-1.0, x_max=1.0):
    # Fixed x range
    x = np.linspace(-20, 20, 1000)
    y = norm.pdf(x, mu, sigma)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')

    # Fill full curve lightly
    ax.fill_between(x, y, color='plum', alpha=0.2)

    # Highlight area between x_min and x_max
    mask = (x >= x_min) & (x <= x_max)
    ax.fill_between(x[mask], y[mask], color='mediumvioletred', alpha=0.6,
                    label=f"P({x_min:.2f} < θ < {x_max:.2f}) ≈ {norm.cdf(x_max, mu, sigma) - norm.cdf(x_min, mu, sigma):.2f}")

    ax.set_title("Probability as Area Under the Curve")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(-20, 20)
    ax.set_ylim(0, max(y) * 1.1)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ"),
    x_min=widgets.FloatSlider(min=-10, max=10, step=0.1, value=-1.0, description="x min"),
    x_max=widgets.FloatSlider(min=-10, max=10, step=0.1, value=1.0, description="x max")
)


interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

<function __main__.plot_gaussian(mu=0.0, sigma=1.0, x_min=-1.0, x_max=1.0)>

**Prior beliefs for theta.** We need to think carefully about what theta means. Theta represents cognitive ability in our go-no-go task. 0 is the worst possible cognitive ability, and 1 is the best. So we should set a prior distribution for theta between 0 and 1. The distributions above do not do this, so let's fix this. We can use a beta distribution to represent our prior belief about theta. The beta distribution is a continuous probability distribution defined on the interval [0, 1], so it is perfect for our needs.


**Exercise.** Play around with beta distribution parameters to see how the shape of the distribution changes:

  - You are complete uncertain about the ability → Set the widest prior

  - You are quite certain that ability will be high  → Set a narrow prior centered on a high value

  - You are quite certain that ability will be low  → Set a narrow prior centered on a low value

  - You are uncertain but you think ability is low  → Set a wide prior centered on a low value

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display, clear_output

# Sliders
n_slider = widgets.IntSlider(value=10, min=1, max=100, step=1, description="n")
k_slider = widgets.IntSlider(value=5, min=0, max=100, step=1, description="k")

# Output area
plot_output = widgets.Output()

# Plot function
def plot_beta(k, n):
    x = np.linspace(0, 1, 500)
    a = k + 1
    b = (n - k) + 1
    y = beta.pdf(x, a, b)

    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.plot(x, y, label=f"Beta({a}, {b})", color='blue')
        ax.fill_between(x, y, color='skyblue', alpha=0.3)
        ax.set_title("Prior beliefs over θ")
        ax.set_xlabel("θ")
        ax.set_ylabel("Probability Density")
        ax.grid(True)
        ax.legend()
        plt.show()

# Unified update function
def on_slider_change(change=None):
    # Avoid recursive triggers by updating value only if needed
    if k_slider.value > n_slider.value:
        k_slider.unobserve(on_slider_change, names='value')
        k_slider.value = n_slider.value
        k_slider.observe(on_slider_change, names='value')
    plot_beta(k_slider.value, n_slider.value)

# Set up observers (after defining callback)
n_slider.observe(on_slider_change, names='value')
k_slider.observe(on_slider_change, names='value')

# Layout and display
display(widgets.VBox([n_slider, k_slider]), plot_output)

# Initial plot
plot_beta(k_slider.value, n_slider.value)


Output()

**Multiplying prior and likelihood.**  We’ve learned that the posterior is proportional to the prior times the likelihood:

$$
\text{Posterior} \propto \text{Prior} \times \text{Likelihood}
$$

The likelihood tells us how well each value of $\theta$ predicted the data. If a value of $\theta$ predicts the data well → the posterior belief increases. If it predicts the data poorly → the posterior belief decreases. So the posterior reshapes your belief, using the likelihood as a reweighting function over your prior.

**Exercise.** Now you can explore this directly: Use the sliders to define a prior distribution and a likelihood. The resulting product is shown as the posterior. By default, this is not normalized (it doesnt inegrate to 1) — you are just seeing the raw shape. This shape still carries meaning: it shows which values of $\theta$ are most consistent with both your prior beliefs and the observed data.
- Set a prior and likelihood.
- Pick a few values of theta, and multiply the prior by the likelihood.
- See if the height of the posterior curve matches your expectation.

**Normalising the Posterior.** You can press the button to normalize the posterior.
To turn the posterior into a true probability distribution, we need to divide by the marginal likelihood — the constant that ensures the posterior integrates to 1 and is therefore a proper probability distribution.

$$
p(\theta \mid \text{data}) = \frac{p(\theta) \cdot p(\text{data} \mid \theta)}{p(\text{data})}
$$

**Exercise.** Click on the button to normalize the posterior. 

- Notice how the area under the curve is now 1, and the posterior is a proper probability distribution.

- The area under the prior and the posterior should be the same. 

- Do you need the same amount of paint to colour each in?

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

# --- Plotting function ---
def plot_bayes(k_prior, n_prior, k_like, n_like, show_norm_post):
    # Prior
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood (visual only)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Unnormalized Posterior
    y_post = y_prior * y_like

    # Normalize if requested
    if show_norm_post:
        norm_const = np.trapz(y_post, x)
        y_post = y_post / norm_const if norm_const > 0 else np.zeros_like(x)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y_prior, label="Prior", color="blue")
    ax.plot(x, y_like, label="Likelihood", color="red")
    ax.plot(x, y_post, label="Posterior", color="green")

    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    ax.set_ylim(0, 10)  # Or another sensible fixed upper limit

    plt.show()

# --- Sliders ---
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior")
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior")

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like")
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like")

# --- Checkbox to toggle normalization
show_norm_post = widgets.Checkbox(value=False, description="Normalize Posterior", indent=False)

# --- Ensure k ≤ n ---
def update_k_max(slider_k, slider_n):
    slider_k.max = slider_n.value
    if slider_k.value > slider_k.max:
        slider_k.value = slider_k.max

def update_k_max_prior(*args):
    update_k_max(k_prior_slider, n_prior_slider)

def update_k_max_like(*args):
    update_k_max(k_like_slider, n_like_slider)

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')
update_k_max_prior()
update_k_max_like()

# --- UI layout ---
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider]),
    show_norm_post
])

# --- Plot binding
out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider,
    'show_norm_post': show_norm_post
})

display(ui, out)


Output()

**Is the Likelihood a proper probability distribution?** Does the likelihood integrate to 1? Not the way we plot it. The likelihood is $p(\mathrm{data} \mid \theta)$: a probability distribution over the data, given a fixed value of $\theta$. It is not a probability distribution over $\theta$. So when we plot $\theta \mapsto p(\mathrm{data} \mid \theta)$, we are not plotting a probability density function — we're plotting a likelihood function, and it does not need to integrate to 1 over $\theta$. However, for each fixed $\theta$, the function is a proper probability distribution over data, and it satisfies:

$$
\int p(\mathrm{data} \mid \theta) \, d\,\mathrm{data} = 1
$$

So yes — the likelihood does integrate to 1, but only over the data, not over $\theta$.

**Exercise.** Wiggle the sliders to get different likelihoods:

- Left, when you plot the likelihood, over different values of theta, you can see that the likelihood does not integrate to 1.

- Right, when you plot the likelihood over different values of data, you can see that the likelihood does integrate to 1.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import ipywidgets as widgets
from IPython.display import display

# --- Plotting function ---
def plot_likelihood_views(n, k, theta_fixed):
    theta_vals = np.linspace(0, 1, 500)
    likelihood_theta = binom.pmf(k, n, theta_vals)
    area_theta = np.trapz(likelihood_theta, theta_vals)

    k_vals = np.arange(0, n + 1)
    likelihood_data = binom.pmf(k_vals, n, theta_fixed)
    area_data = np.sum(likelihood_data)

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Left: likelihood as function of θ
    axes[0].plot(theta_vals, likelihood_theta, color='black', lw=2)
    axes[0].set_title(rf"$\theta \mapsto p(k={k} \mid \theta, n={n})$")
    axes[0].set_xlabel("θ")
    axes[0].set_ylabel(r"$p(k \mid \theta, n)$")
    axes[0].grid(True, linestyle='--', alpha=0.5)
    axes[0].set_ylim(0, 1)
    axes[0].text(0.95, 0.95, f"∫ dθ ≈ {area_theta:.3f}", transform=axes[0].transAxes,
                 ha='right', va='top', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray'))

    # Right: likelihood as function of data (k)
    axes[1].bar(k_vals, likelihood_data, color='black', alpha=0.8)
    axes[1].set_title(rf"$k \mapsto p(k \mid \theta={theta_fixed:.2f}, n={n})$")
    axes[1].set_xlabel("k (number of successes)")
    axes[1].set_ylabel("Probability")
    axes[1].grid(True, linestyle='--', alpha=0.5)
    axes[1].set_ylim(0, 1)
    axes[1].text(0.95, 0.95, f"∑ = {area_data:.3f}", transform=axes[1].transAxes,
                 ha='right', va='top', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray'))

    plt.tight_layout()
    plt.show()

# --- Sliders ---
n_slider = widgets.IntSlider(min=1, max=50, value=10, step=1, description="n (trials)")
k_slider = widgets.IntSlider(min=0, max=50, value=5, step=1, description="k (successes)")
theta_slider = widgets.FloatSlider(min=0.01, max=0.99, value=0.5, step=0.01, description="θ (fixed)")

# --- Enforce k ≤ n ---
def update_k_max(*args):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

n_slider.observe(update_k_max, names='value')
update_k_max()

# --- Layout ---
ui = widgets.VBox([
    widgets.HTML("Left: Likelihood as a function of θ"),
    widgets.HTML("Right: Likelihood as a function of data"),
    widgets.HBox([n_slider, k_slider, theta_slider])
])

out = widgets.interactive_output(plot_likelihood_views, {
    'n': n_slider,
    'k': k_slider,
    'theta_fixed': theta_slider
})

display(ui, out)


Output()

**Bayesian credibility intervals.** A Bayesian credibility interval is a range of values that contains the true value of the parameter with a certain probability. 
It is similar to a confidence interval in frequentist statistics, but it is based on the posterior distribution of the parameter rather than the sampling distribution. Its interpretation is actually what most people think of when they hear the term "confidence interval". Its what they are looking for when they ask for a confidence interval. You can construct a Bayesian credibility interval by picking the probability you want to contain the true value of the parameter, and then finding the range of values that contains that probability. 

**Exercise.** Click on the code cell below and press play to run. 

- Play with the sliders to see how the credibility interval changes as you change the probability:

- The most typical is the 95% credibility interval $BCI_{95}$, which contains the value of the parameter with 95% probability. 

- The 50% credibility interval $BCI_{50}$ would contains the value of the parameter with 50% probability. And so on.

- How is this different from the frequentist confidence interval? 

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian_bci(mu=0.0, sigma=1.0, bci=95.0):
    # Fixed x and y ranges
    x = np.linspace(-10, 10, 1000)
    y = norm.pdf(x, mu, sigma)

    # Compute credible interval bounds
    alpha = 1 - bci / 100
    lower_bound = norm.ppf(alpha / 2, mu, sigma)
    upper_bound = norm.ppf(1 - alpha / 2, mu, sigma)
    y_bound = norm.pdf([lower_bound, upper_bound], mu, sigma).min()

    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot curve and shading
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')
    ax.fill_between(x, y, color='plum', alpha=0.2)

    # Highlight credible interval
    mask = (x >= lower_bound) & (x <= upper_bound)
    ax.fill_between(x[mask], y[mask], color='mediumvioletred', alpha=0.6,
                    label=f"{bci:.0f}% BCI: [{lower_bound:.2f}, {upper_bound:.2f}]")

    # Dashed lines at interval bounds
    ax.hlines(y=y_bound, xmin=lower_bound, xmax=upper_bound,
              color='black', linestyle='--', linewidth=1)
    ax.vlines([lower_bound, upper_bound], ymin=0, ymax=y_bound,
              color='black', linestyle='--', linewidth=1)

    # Fixed axes
    ax.set_xlim(-10, 10)
    ax.set_ylim(0, 0.5)  # Fixed y-range to accommodate all practical normal densities

    # Labels and legend
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.legend(loc="upper left")
    plt.show()

# Interactive sliders
interact(
    plot_gaussian_bci,
    mu=widgets.FloatSlider(min=-5, max=5, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description="σ"),
    bci=widgets.FloatSlider(min=50, max=99, step=1, value=95, description="BCI (%)")
)


interactive(children=(FloatSlider(value=0.0, description='μ', max=5.0, min=-5.0), FloatSlider(value=1.0, descr…

<function __main__.plot_gaussian_bci(mu=0.0, sigma=1.0, bci=95.0)>

**Summarising the posterior.** With a proper normalised posterior, we may want to summarise it with credible intervals. These are the Bayesian equivalent of confidence intervals. But better, obvs. 

**Exercise.** Play with the slider that finds the Xth percentile credible interval.

- What happens to the interval when you drop the percentile lower?

- Plot the MAP - Maximum a posteriori. What is it? 

- How would you define it?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.special import betaln
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_bayes(k_prior, n_prior, k_like, n_like, ci_width, show_shading, show_mean, show_map, show_curves):
    # Prior parameters
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood as Beta PDF (for visualization only)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Posterior parameters (from conjugate Beta-Binomial update)
    a_post = a_prior + k_like
    b_post = b_prior + n_like - k_like
    y_post = beta.pdf(x, a_post, b_post)

    # Compute proper marginal likelihood
    log_ml = betaln(k_like + a_prior, n_like - k_like + b_prior) - betaln(a_prior, b_prior)
    marginal_likelihood = np.exp(log_ml)

    # Compute Bayesian credible interval (BCI)
    lower = beta.ppf((1 - ci_width / 100) / 2, a_post, b_post)
    upper = beta.ppf(1 - (1 - ci_width / 100) / 2, a_post, b_post)

    # Posterior Mean and MAP
    posterior_mean = a_post / (a_post + b_post)
    map_index = np.argmax(y_post)
    posterior_map = x[map_index]

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 6))
    
    if show_curves:
        ax.plot(x, y_prior, label="Prior", color="blue")
        ax.plot(x, y_like, label="Likelihood (visualized)", color="red")

    ml_text = f"Posterior (marginal likelihood ≈ {marginal_likelihood:.3f})"
    ax.plot(x, y_post, label=ml_text, color="green")

    if show_shading:
        ax.fill_between(x, y_post, where=(x >= lower) & (x <= upper), color='gray', alpha=0.3, label=f"{ci_width}% BCI")
    else:
        ax.axvline(lower, color='gray', linestyle='--', label=f"{ci_width}% BCI")
        ax.axvline(upper, color='gray', linestyle='--')

    if show_mean:
        ax.axvline(posterior_mean, color='black', linestyle=':', label="Posterior Mean")

    if show_map:
        ax.axvline(posterior_map, color='purple', linestyle='-.', label="Posterior MAP")

    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# Sliders
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

ci_slider = widgets.IntSlider(min=50, max=99, step=1, value=95, description="BCI (%)", style={'description_width': 'initial'})

# Toggles
shading_toggle = widgets.Checkbox(value=False, description='Shade BCI')
mean_toggle = widgets.Checkbox(value=False, description='Show Posterior Mean')
map_toggle = widgets.Checkbox(value=False, description='Show MAP')
curves_toggle = widgets.Checkbox(value=False, description='Show Prior & Likelihood')

# Reusable fix for k ≤ n
def enforce_k_leq_n(k_slider, n_slider):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

# Observers
def update_k_max_prior(*args):
    enforce_k_leq_n(k_prior_slider, n_prior_slider)

def update_k_max_like(*args):
    enforce_k_leq_n(k_like_slider, n_like_slider)

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# Initial sync
update_k_max_prior()
update_k_max_like()

# Layout
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider]),
    widgets.HTML("<b>Posterior Display Options</b>"),
    ci_slider,
    shading_toggle,
    mean_toggle,
    map_toggle,
    curves_toggle
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider,
    'ci_width': ci_slider,
    'show_shading': shading_toggle,
    'show_mean': mean_toggle,
    'show_map': map_toggle,
    'show_curves': curves_toggle
})

display(ui, out)


Output()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.special import betaln
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

# --- Plotting function ---
def plot_bayes(k_prior, n_prior, k_like, n_like, ci_width, show_shading, show_mean, show_map, show_curves):
    # Prior parameters
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood as Beta PDF (for visualization only)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Posterior parameters (from conjugate Beta-Binomial update)
    a_post = a_prior + k_like
    b_post = b_prior + n_like - k_like
    y_post = beta.pdf(x, a_post, b_post)

    # Compute proper marginal likelihood
    log_ml = betaln(k_like + a_prior, n_like - k_like + b_prior) - betaln(a_prior, b_prior)
    marginal_likelihood = np.exp(log_ml)

    # Compute Bayesian credible interval (BCI)
    lower = beta.ppf((1 - ci_width / 100) / 2, a_post, b_post)
    upper = beta.ppf(1 - (1 - ci_width / 100) / 2, a_post, b_post)

    # Posterior Mean and MAP
    posterior_mean = a_post / (a_post + b_post)
    map_index = np.argmax(y_post)
    posterior_map = x[map_index]

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 6))
    
    if show_curves:
        ax.plot(x, y_prior, label="Prior", color="blue")
        ax.plot(x, y_like, label="Likelihood (visualized)", color="red")

    ml_text = f"Posterior (marginal likelihood ≈ {marginal_likelihood:.3f})"
    ax.plot(x, y_post, label=ml_text, color="green")

    if show_shading:
        ax.fill_between(x, y_post, where=(x >= lower) & (x <= upper), color='gray', alpha=0.3, label=f"{ci_width}% BCI")
    else:
        ax.axvline(lower, color='gray', linestyle='--', label=f"{ci_width}% BCI")
        ax.axvline(upper, color='gray', linestyle='--')

    if show_mean:
        ax.axvline(posterior_mean, color='black', linestyle=':', label="Posterior Mean")

    if show_map:
        ax.axvline(posterior_map, color='purple', linestyle='-.', label="Posterior MAP")

    ax.set_title("Bayesian Updating with BCI, Mean, and MAP")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# Sliders
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

ci_slider = widgets.IntSlider(min=50, max=99, step=1, value=95, description="BCI (%)", style={'description_width': 'initial'})

# Toggles (defaults set to False)
shading_toggle = widgets.Checkbox(value=False, description='Shade BCI')
mean_toggle = widgets.Checkbox(value=False, description='Show Posterior Mean')
map_toggle = widgets.Checkbox(value=False, description='Show MAP')
curves_toggle = widgets.Checkbox(value=False, description='Show Prior & Likelihood')

# Reusable fix for k ≤ n
def enforce_k_leq_n(k_slider, n_slider):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

# Observers
def update_k_max_prior(*args):
    enforce_k_leq_n(k_prior_slider, n_prior_slider)

def update_k_max_like(*args):
    enforce_k_leq_n(k_like_slider, n_like_slider)

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# Initial sync
update_k_max_prior()
update_k_max_like()

# Layout
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider]),
    widgets.HTML("<b>Posterior Display Options</b>"),
    ci_slider,
    shading_toggle,
    mean_toggle,
    map_toggle,
    curves_toggle
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider,
    'ci_width': ci_slider,
    'show_shading': shading_toggle,
    'show_mean': mean_toggle,
    'show_map': map_toggle,
    'show_curves': curves_toggle
})

display(ui, out)


Output()

**Compute the posterior by updating the beta.** The beta distribution allows for a very simple way to compute the posterior just by adding n and k to the inputs to the Beta function. This is called computing the posterior analytically, and it is a special case of the *conjugate* prior. This simplicity is, alas, not always possible.

**Exercise.** Click on the code cell below and press play to run.

- Play with the sliders to see how the posterior changes as you change the number of successes and failures.

- Notice that the posterior is another beta distribution, with parameters $n + k$ and $m + n - k$.

- This makes it easy to compute the posterior. 

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display, HTML

x = np.linspace(0, 1, 1000)

def plot_special_case(n, k):
    if k > n:
        print("Error: k must be ≤ n.")
        return

    # Prior: Beta(1, 1)
    a_prior, b_prior = 1, 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Posterior: Beta(1 + k, 1 + n - k)
    a_post = 1 + k
    b_post = 1 + (n - k)
    y_post = beta.pdf(x, a_post, b_post)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot prior
    ax.plot(x, y_prior, label="Prior: Beta(1, 1)", color="blue")
    ax.fill_between(x, y_prior, color="skyblue", alpha=0.4)

    # Plot posterior
    full_label = f"Posterior: Beta(1 + k, 1 + (n − k)) = Beta({a_post}, {b_post})"
    ax.plot(x, y_post, label=full_label, color="green")
    ax.fill_between(x, y_post, color="lightgreen", alpha=0.4)

    # Adjust layout and axis
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_ylim(0, max(np.max(y_post), np.max(y_prior)) * 1.2)
    ax.grid(True)

    # Move legend outside to the right
    fig.subplots_adjust(right=0.75)
    ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=False)

    plt.show()

# Sliders
n_slider = widgets.IntSlider(min=1, max=100, step=1, value=56, description="n (trials)", style={'description_width': 'initial'})
k_slider = widgets.IntSlider(min=0, max=56, step=1, value=43, description="k (correct)", style={'description_width': 'initial'})

# Auto-adjust k bounds
def update_k_slider(*args):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

n_slider.observe(update_k_slider, names='value')
update_k_slider()  # Initial sync

# Layout
ui = widgets.VBox([
    widgets.HBox([n_slider, k_slider])
])

out = widgets.interactive_output(plot_special_case, {'n': n_slider, 'k': k_slider})

display(ui, out)


Output()

**Sequential vs aggregated updating.**  Bayesian updating can be done in two ways. You can update your beliefs step by step as new data arrives (sequential updating), or you can combine all the data and update in one go (aggregated updating). With conjugate priors like the Beta distribution, both methods give the same result. Let’s see how this works in a simple example.

We’ll start with a uniform prior: Beta(1, 1). You then observe two datasets:
- First: 9 correct out of 10 (k = 9, n = 10)
- Second: 3 correct out of 5 (k = 3, n = 5)

There are two ways to compute the posterior:

**Sequential updating**  
   - Start with Beta(1, 1)  
   - Update with the first dataset → Beta(10, 2)  
   - Use that as the new prior, update with the second dataset → Beta(13, 4)

**Aggregated updating**  
   - Combine the data: k = 12, n = 15  
   - Start with Beta(1, 1), update once → Beta(13, 4)


**Exercise.** Use the sliders to try both methods and confirm they give the same result.

- First, try the sequential approach:  
  Set prior to Beta(1, 1), then enter k = 9, n = 10.  
  Use the resulting posterior as the new prior, then enter k = 3, n = 5.

- Then try the aggregated approach:  
  Set prior to Beta(1, 1), then enter k = 12, n = 15.

- Do you get the same posterior in both cases?



In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_update(n_prior, k_prior, n_new, k_new):
    if k_prior > n_prior or k_new > n_new:
        print("Error: k must be ≤ n.")
        return

    # Prior = Beta(1 + k_prior, 1 + n_prior - k_prior)
    a_prior = 1 + k_prior
    b_prior = 1 + (n_prior - k_prior)
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Posterior = Beta(1 + total_k, 1 + total_n - total_k)
    total_k = k_prior + k_new
    total_n = n_prior + n_new
    a_post = 1 + total_k
    b_post = 1 + (total_n - total_k)
    y_post = beta.pdf(x, a_post, b_post)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))

    ax.plot(x, y_prior, label=f"Prior: Beta(1 + {k_prior}, 1 + {n_prior - k_prior})", color="blue")
    ax.fill_between(x, y_prior, color="skyblue", alpha=0.4)

    ax.plot(x, y_post, label=f"Posterior: Beta(1 + {total_k}, 1 + {total_n - total_k})", color="green")
    ax.fill_between(x, y_post, color="lightgreen", alpha=0.4)

    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_ylim(0, max(np.max(y_prior), np.max(y_post)) * 1.2)
    ax.grid(True)

    # Adjust spacing and move legend to right
    fig.subplots_adjust(right=0.75)
    ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=False)

    plt.show()

# Sliders
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=20, description="n_prior")
k_prior_slider = widgets.IntSlider(min=0, max=20, step=1, value=10, description="k_prior")

n_new_slider = widgets.IntSlider(min=0, max=100, step=1, value=30, description="n_new")
k_new_slider = widgets.IntSlider(min=0, max=30, step=1, value=15, description="k_new")

# Auto-limit k sliders
def update_k_prior_max(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_new_max(*args):
    k_new_slider.max = n_new_slider.value
    if k_new_slider.value > k_new_slider.max:
        k_new_slider.value = k_new_slider.max

n_prior_slider.observe(update_k_prior_max, names='value')
n_new_slider.observe(update_k_new_max, names='value')

# Initial sync
update_k_prior_max()
update_k_new_max()

# Layout
ui = widgets.VBox([
    widgets.HTML("🔵 <b>Prior</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("🟢 <b>New Data</b>"),
    widgets.HBox([n_new_slider, k_new_slider])
])

out = widgets.interactive_output(plot_update, {
    'n_prior': n_prior_slider,
    'k_prior': k_prior_slider,
    'n_new': n_new_slider,
    'k_new': k_new_slider
})

display(ui, out)


Output()

**MCMC.** To illustrate the power of MCMC, we demo with this interactive example. The demo runs JAGS via python. JAGS is a sampler that implements MCMC, though there are many samplers available. The model is set up via as specific language. It is everything inside "model {...}" in the code cell below. Here the model is a simple Bernoulli model, where we have a prior on theta, and we have a likelihood for observe k successes out of n trials.

**Exercise.** Play with the data k and n and set the number of samples that sampler runs. Keep it low to begin with.

-  Compare sampling to an analytical posterior for the same data. Click on the show analytic posteror button to overlay the analytical posterior. 

- How do you get the MCMC posterior to better match the analytical posterior? 

In [38]:
import pyjags
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import beta

# Ensure plots show inline
%matplotlib inline

def run_jags(k=6, n=10, samples=100, show_analytic=False):
    if k > n:
        print("k must be ≤ n")
        return

    # Data setup
    y = np.array([1]*k + [0]*(n-k))

    # JAGS model string
    model_code = """
    model {
      for (i in 1:N) {
        y[i] ~ dbern(theta)
      }
      theta ~ dbeta(1, 1)
    }
    """

    data = {"y": y, "N": len(y)}
    inits = [{"theta": 0.5}]

    # Run JAGS model
    model = pyjags.Model(code=model_code, data=data, init=inits, chains=1)
    model.update(100)  # Burn-in
    result = model.sample(samples, vars=["theta"])
    theta_samples = result["theta"].reshape(-1)

    # Plotting
    fig, axs = plt.subplots(1, 3, figsize=(15, 4))

    # --- Trace plot ---
    axs[0].plot(theta_samples)
    axs[0].set_title("Trace of θ (Chain 1)")
    axs[0].set_xlabel("Sample")
    axs[0].set_ylabel("θ")
    axs[0].set_ylim(0, 1)
    axs[0].grid(True)

    # --- Horizontal histogram ---
    axs[1].hist(theta_samples, bins=30, density=True, alpha=0.6, color='gray', orientation='horizontal', label='MCMC posterior')
    axs[1].set_title("Posterior of θ")
    axs[1].set_xlabel("Density")
    axs[1].set_ylabel("θ")
    axs[1].set_ylim(0, 1)

    # --- Line-over-histogram plot ---
    axs[2].hist(theta_samples, bins=30, density=True, alpha=0.5, color='steelblue', label='MCMC posterior')

    # Optional analytical overlay
    if show_analytic:
        alpha_post = 1 + k
        beta_post = 1 + (n - k)
        theta_vals = np.linspace(0, 1, 200)
        analytical_posterior = beta.pdf(theta_vals, alpha_post, beta_post)
        axs[1].plot(analytical_posterior, theta_vals, 'r-', lw=2, label='Analytical posterior')
        axs[1].legend()

        axs[2].plot(theta_vals, analytical_posterior, 'r-', lw=2, label='Analytical posterior')
        axs[2].legend()

    axs[2].set_title(f"Posterior of θ (k={k}, n={n})")
    axs[2].set_xlabel("θ")
    axs[2].set_ylabel("Density")
    axs[2].set_xlim(0, 1)
    axs[2].grid(True)

    plt.tight_layout()
    plt.show()

# Widgets
k_slider = widgets.IntSlider(value=6, min=0, max=20, step=1, description='Successes (k):')
n_slider = widgets.IntSlider(value=10, min=1, max=20, step=1, description='Trials (n):')
sample_slider = widgets.IntSlider(value=100, min=100, max=10000, step=100, description='Samples:')

analytic_toggle = widgets.ToggleButton(
    value=False,
    description='Show analytic posterior',
    tooltip='Toggle analytical posterior overlay',
    icon='line-chart'
)

# Link function to widgets
ui = widgets.VBox([k_slider, n_slider, sample_slider, analytic_toggle])
out = widgets.interactive_output(run_jags, {
    'k': k_slider,
    'n': n_slider,
    'samples': sample_slider,
    'show_analytic': analytic_toggle
})

# Display
display(ui, out)


Output()

---

## Lecture 3

**Binomial distribution.**  Here we can see the binomial distribution: the probability of getting `k` successes in `n` independent trials, where each trial has a success probability of `θ`. Use the sliders below to adjust `n` and `θ`, and watch how the shape of the distribution changes. The equation updates to show how the probability is computed based on the current values.

**Exercise.** Play with the sliders and answer the following:

- What happens when θ is close to 0 or close to 1?

- How does the distribution change as you increase n?

- When is the distribution symmetric, and when is it skewed?

- What value of `k` seems most likely, and how does that relate to `θ × n`?

Try predicting the outcome before you move the sliders — then use the plot to check your intuition.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Math, Latex, clear_output

# Define sliders
theta_slider = widgets.FloatSlider(value=0.5, min=0.01, max=0.99, step=0.01, description='θ')
n_slider = widgets.IntSlider(value=10, min=1, max=100, step=1, description='n')

# Define function to update the plot and LaTeX
def update_plot(theta, n):
    clear_output(wait=True)
    
    # Binomial support and PMF
    k = np.arange(0, n+1)
    pmf = [np.math.comb(n, ki) * theta**ki * (1 - theta)**(n - ki) for ki in k]
    
    # Display formula with current values
    display(Math(f"P(k\\mid\\theta={theta:.2f},\\ n={n}) = \\binom{{n}}{{k}} \\theta^k (1 - \\theta)^{{n - k}}"))
    
    # Plot
    plt.figure(figsize=(8, 4))
    plt.bar(k, pmf, color='skyblue', edgecolor='black')
    plt.title(f'Binomial Distribution: n={n}, θ={theta:.2f}')
    plt.xlabel('k')
    plt.ylabel('P(k | θ, n)')
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

# Interactive output
widgets.interact(update_plot, theta=theta_slider, n=n_slider)


interactive(children=(FloatSlider(value=0.5, description='θ', max=0.99, min=0.01, step=0.01), IntSlider(value=…

<function __main__.update_plot(theta, n)>

**MCMC convergaence checks.** This is the same model as the one we used in the MCMC demo. The model is a simple Bernoulli model, where we have a prior on theta, and we have a likelihood for observe k successes out of n trials. The model is set up via as specific language. It is everything inside "model {...}" in the code cell below. We run it here so that we can look at the model outputs and see if they look ok. 

**Exercise.** Play with the sliders, run the model, and check the convergence diagnostics.
- Do the chains look like they have converged?
- Do they look like "hairy caterpillars"?
- Is the R-hat statistic close to 1?
- Is the Pope Catholic?
- Just checking you are awake. x 


In [18]:
import pyjags
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Math, Latex, clear_output

# Define the JAGS model as a string
model_code = """
model {
  theta ~ dbeta(1,1)
  k ~ dbin(theta, n)
}
"""

# R-hat computation
def compute_rhat(chains):
    m = len(chains)
    n = len(chains[0])
    
    chain_means = np.array([np.mean(c) for c in chains])
    chain_vars = np.array([np.var(c, ddof=1) for c in chains])

    B = n * np.var(chain_means, ddof=1)
    W = np.mean(chain_vars)
    var_hat = ((n - 1) / n) * W + (1 / n) * B
    Rhat = np.sqrt(var_hat / W)

    return Rhat, B, W, var_hat, chain_means, chain_vars

# Main model runner
def run_model(k, n, samples):
    clear_output(wait=True)
    print(f"Running with k={k}, n={n}, samples={samples} per chain\n")
    
    chains = []
    for i in range(4):
        data = {'k': k, 'n': n}
        model = pyjags.Model(code=model_code, data=data, chains=1, adapt=500)
        model.update(1000)
        result = model.sample(samples, vars=['theta'])
        theta_samples = result['theta'].reshape(-1)
        chains.append(theta_samples)
        plt.plot(theta_samples, label=f"Chain {i+1}")

    plt.title("Trace plots of θ for 4 chains")
    plt.xlabel("Sample")
    plt.ylabel("θ")
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Compute R-hat
    Rhat, B, W, var_hat, chain_means, chain_vars = compute_rhat(chains)

    # Print results
    print("Per-chain statistics:")
    for i, (mean, var) in enumerate(zip(chain_means, chain_vars)):
        print(f"  Chain {i+1}: mean = {mean:.4f}, var = {var:.5f}")

    print(f"\nBetween-chain variance (B): {B:.5f}")
    print(f"Within-chain variance (W): {W:.5f}")
    print(f"Estimated variance (var_hat): {var_hat:.5f}")
    print(f"R-hat: {Rhat:.4f}\n")

    display(Math(r"""
    \hat{R} = \sqrt{ \frac{ \left( \frac{n-1}{n} \right) W + \left( \frac{1}{n} \right) B }{W} }
    """))

# Interactive widgets
k_slider = widgets.IntSlider(value=6, min=0, max=20, description='Successes (k)')
n_slider = widgets.IntSlider(value=10, min=1, max=20, description='Trials (n)')
samples_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Samples')

run_button = widgets.Button(description="Run model")
output = widgets.Output()

def on_click(b):
    with output:
        run_model(k_slider.value, n_slider.value, samples_slider.value)

run_button.on_click(on_click)
display(widgets.VBox([k_slider, n_slider, samples_slider, run_button, output]))


**Inferring the difference between two rates.** In this model, we observe two processes — for example, two groups or experimental conditions — each with a number of successes out of a number of trials. We assume the underlying success rates are governed by parameters $\theta_1$ and $\theta_2$, with uniform Beta priors:

$\theta_1 \sim \text{Beta}(1, 1)$, $\theta_2 \sim \text{Beta}(1, 1)$  
$k_1 \sim \text{Binomial}(n_1, \theta_1)$, $k_2 \sim \text{Binomial}(n_2, \theta_2)$

Our quantity of interest is the difference in success rates:

$\delta = \theta_1 - \theta_2$

This tells us how much more (or less) likely success is in one group than the other.

Use the sliders to adjust $k_1$, $n_1$, $k_2$, and $n_2$. The trace plots show MCMC samples of $\delta$, and the histogram shows its posterior distribution. You can also overlay an approximate analytical solution for comparison.

**Exercise.** Explore the model using the sliders and answer the following:

- When does the posterior of $\delta$ look symmetric? When is it skewed?
- How does increasing $n_1$ or $n_2$ affect the width of the posterior?
- What happens when $k_1 = k_2$ but $n_1 \ne n_2$?
- Use the checkbox to compare the MCMC posterior with the analytical approximation. How close are they?

Try predicting what the posterior will look like before adjusting the sliders — then use the plot to check your intuition.


In [ ]:
import pyjags
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Math, clear_output

# JAGS model code
model_code = """
model {
  k1 ~ dbin(theta1, n1)
  k2 ~ dbin(theta2, n2)
  theta1 ~ dbeta(1,1)
  theta2 ~ dbeta(1,1)
  delta <- theta1 - theta2
}
"""

# R-hat calculation
def compute_rhat(chains):
    m = len(chains)
    n = len(chains[0])
    chain_means = np.array([np.mean(c) for c in chains])
    chain_vars = np.array([np.var(c, ddof=1) for c in chains])
    B = n * np.var(chain_means, ddof=1)
    W = np.mean(chain_vars)
    var_hat = ((n - 1) / n) * W + (1 / n) * B
    Rhat = np.sqrt(var_hat / W)
    return Rhat, B, W, var_hat, chain_means, chain_vars

# Main model function
def run_model(k1, n1, k2, n2, samples):
    clear_output(wait=True)
    print(f"Running with (k1={k1}, n1={n1}), (k2={k2}, n2={n2}), samples={samples} per chain")

    data = {"k1": k1, "n1": n1, "k2": k2, "n2": n2}
    chains = []

    for i in range(4):
        model = pyjags.Model(code=model_code, data=data, chains=1, adapt=500)
        model.update(1000)
        result = model.sample(samples, vars=["delta"])
        delta_chain = result["delta"].reshape(-1)
        chains.append(delta_chain)

    # Plot traces and histogram
    fig = plt.figure(figsize=(12, 6))
    grid = plt.GridSpec(1, 2, width_ratios=[3, 1], wspace=0.3)

    ax_traces = fig.add_subplot(grid[0])
    for i in range(4):
        ax_traces.plot(chains[i], label=f"Chain {i+1}")
    ax_traces.set_title("Trace plots of δ (4 chains)")
    ax_traces.set_xlabel("Sample")
    ax_traces.set_ylabel("δ = θ₁ − θ₂")
    ax_traces.axhline(0, color='black', linestyle='--', lw=1)
    ax_traces.legend()
    ax_traces.grid(True)

    ax_hist = fig.add_subplot(grid[1])
    all_samples = np.concatenate(chains)
    ax_hist.hist(all_samples, bins=30, orientation='horizontal', density=True, color='gray', alpha=0.6, label='Posterior')
    ax_hist.set_title("Posterior of δ")
    ax_hist.set_xlabel("Density")
    ax_hist.set_ylabel("δ")
    ax_hist.grid(True)

    plt.tight_layout()
    plt.show()

    # R-hat diagnostics
    Rhat, B, W, var_hat, chain_means, chain_vars = compute_rhat(chains)

    print("Per-chain statistics:")
    for i, (mean, var) in enumerate(zip(chain_means, chain_vars)):
        print(f"  Chain {i+1}: mean = {mean:.4f}, var = {var:.5f}")

    print(f"\nBetween-chain variance (B): {B:.5f}")
    print(f"Within-chain variance (W): {W:.5f}")
    print(f"Estimated variance (var_hat): {var_hat:.5f}")
    print(f"R-hat: {Rhat:.4f}\n")

    display(Math(r"""
    \hat{R} = \sqrt{ \frac{ \left( \frac{n-1}{n} \right) W + \left( \frac{1}{n} \right) B }{W} }
    """))

# Sliders
k1_slider = widgets.IntSlider(value=15, min=0, max=30, description='k₁')
n1_slider = widgets.IntSlider(value=20, min=1, max=40, description='n₁')
k2_slider = widgets.IntSlider(value=10, min=0, max=30, description='k₂')
n2_slider = widgets.IntSlider(value=20, min=1, max=40, description='n₂')
samples_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Samples')

run_button = widgets.Button(description="Run model")
output = widgets.Output()

def on_click(b):
    with output:
        run_model(
            k1_slider.value, n1_slider.value,
            k2_slider.value, n2_slider.value,
            samples_slider.value
        )

run_button.on_click(on_click)

display(widgets.VBox([
    k1_slider, n1_slider, k2_slider, n2_slider,
    samples_slider, run_button, output
]))
